# Final Project - Nolan & Calvin

https://colab.research.google.com/drive/1M_ghrBpds2rQkPfJW9V6vCRifFLJzDbv?usp=sharing CLICK HERE TO VIEW ALL OUIPUTS!!! - NOLAN

In [ ]:
!pip install -q amplpy ampltools pandas

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "a3cb9024-0b17-4508-ae2b-528e312a26ec"

from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6378.6763 expiring 20240531: INFO 645 Prescriptive Analytics, Atma Iyer, Virginia Commonwealth University.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import sys

#Import the sheet 'TestSKU' onto sku_data df
sku_data = pd.read_excel("/content/drive/MyDrive/INFO645 Prescriptive Analytics/Datasets/Interchange Group Data - Shared.xlsx",
                       sheet_name="SKUData",
                        index_col=0)

#Let SKUs = the set of SKUs in the SKU portfolio
sku = list(sku_data.index)
print('SKUs = ', sku)

#Let min_TimeToFreeze[i] be the minimum freeze time for an SKU i, i in sku
min_TimeToFreeze = dict(zip(sku,sku_data.Min_freeze_time))
print('Min Time to Freeze for each SKU = ', min_TimeToFreeze)


#Import the sheet 'TestData' onto trans_data df
cell_data = pd.read_excel("/content/drive/MyDrive/INFO645 Prescriptive Analytics/Datasets/Interchange Group Data - Shared.xlsx",
                       sheet_name="CellData")

#Let C = the set of cells in the freezer
c = list(cell_data.Cells)
print('Cells = ', c)

#Let rem_TimetoFreeze[i] = the remaining time to freeze whatever is in cell i currently, i in c
rem_TimeToFreeze = dict(zip(c,cell_data.Rem_TimeToFreeze))
print('Remaining time to freeze in a cell = ', rem_TimeToFreeze) #This is set to 0 for the prototype

#Import the sheet 'TestData' onto trans_data df
pallet_data = pd.read_excel("/content/drive/MyDrive/INFO645 Prescriptive Analytics/Datasets/Interchange Group Data - Shared.xlsx",
                       sheet_name="PalletData")

#Let pallets = the set of pallets to be loaded in each cell in the freezer
pallets = list(pallet_data.Pallets)
print('Pallets = ', pallets)

#Let pallet_sku[i] = the SKU in a pallet i, i in pallet
pallet_sku = dict(zip(pallets, pallet_data.Pallet_SKU))
print('The SKU in each pallet = ', pallet_sku)

#Let pallet_TimeToFreeze[i] = the time it takes the content of pallet i to freeze, i in pallets
pallet_TimeToFreeze = dict(zip(pallets, pallet_data.Pallet_TimeToFreeze))
print('Time each pallet having each SKU takes to freeze = ', pallet_TimeToFreeze)

SKUs =  [1301, 1306, 1511, 1521, 1552, 1710, 1752, 5000, 5012, 5013, 5021, 5025, 5026, 5027, 5028, 5030, 5315, 5040, 5055, 5064, 5070, 5071, 5072, 5073, 5074, 5075, 5076, 5077, 5078, 5079, 5110, 5111, 5116, 5119, 5120, 5122, 5211, 5309, 5310, 5311, 5330, 5331, 5332, 5333, 5334, 5335, 5336, 5337, 5338, 5339, 5406, 5455, 5509, 5510, 5519, 5527, 5528, 5533, 5605, 5606, 5650, 5651, 5652, 5653, 5701, 5702, 5703, 5710, 5711, 5712, 5715, 8003, 8006, 8523, 10018, 11018, 11217, 11418, 12218, 13018, 14017, 14217, 14220, 14417, 14617, 14620, 14917, 15017, 15117, 15319, 16018, 30128, 31328, 31528, 31928, 32228, 33128, 34627, 34828, 35027, 35028, 36428, 39728, 40028, 40328, 42828, 43028, 43127, 43227, 43328, 44728, 44838, 49128, 49238, 49278, 49428, 49538, 49828, 31227, 33198, 42328, 42398, 47898, 49228, 49628]
Min Time to Freeze for each SKU =  {1301: 51.25, 1306: 27.75, 1511: 46.0, 1521: 51.25, 1552: 51.25, 1710: 51.25, 1752: 51.25, 5000: 51.25, 5012: 51.25, 5013: 51.25, 5021: 51.25, 5025: 46.0, 

In [ ]:
ampl.eval ('''

reset;

set sku;
set c;
set pallets;

param min_TimeToFreeze{sku};
param rem_TimeToFreeze{c};
param pallet_sku{pallets};
param pallet_TimeToFreeze{pallets};

var x{pallets, c} binary;
var all_cells_filled binary;
var pallet_assigned{pallets} binary;  # Whether a pallet is assigned to any cell

minimize Time_Objective:
  sum {p in pallets, cell in c} (pallet_TimeToFreeze[p] + rem_TimeToFreeze[cell]) * x[p, cell];

subject to CellCapacity{cell in c}:
  sum{p in pallets} x[p,cell] <=1;

subject to PalletAssignment{p in pallets}:
    sum {cell in c} x[p, cell] = pallet_assigned[p];

subject to MaximizeCellUsage:
    sum {cell in c} sum {p in pallets} x[p, cell] >= min(card(c), card(pallets));

''')

In [ ]:
ampl.set['sku'] = sku
ampl.set['c'] = c
ampl.set['pallets'] = pallets
ampl.param['min_TimeToFreeze'] = min_TimeToFreeze
ampl.param['rem_TimeToFreeze'] = rem_TimeToFreeze
ampl.param['pallet_sku'] = pallet_sku
ampl.param['pallet_TimeToFreeze'] = pallet_TimeToFreeze

In [ ]:
ampl.setOption('solver', 'gurobi')
ampl.solve()

Gurobi 11.0.1: Gurobi 11.0.1: optimal solution; objective 10370.21
5422 simplex iterations
1 branching node


In [ ]:
ampl.eval('''expand;''')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# After solving the model
#ampl.eval('''display x;''')

# Extract the solution for displaying
solution_df = pd.DataFrame([(p, c) for (p, c), v in ampl.getVariable('x').getValues().toDict().items() if v == 1], columns=['Pallets', 'Cells'])

# Convert columns to string if necessary for consistent data types
solution_df['Pallets'] = solution_df['Pallets'].astype(str)
solution_df['Cells'] = solution_df['Cells'].astype(str)
pallet_data['Pallets'] = pallet_data['Pallets'].astype(str)
cell_data['Cells'] = cell_data['Cells'].astype(str)

# Join with pallet and cell data for full information
solution_df = solution_df.merge(pallet_data[['Pallets', 'Pallet_SKU', 'Pallet_TimeToFreeze']], on='Pallets')
solution_df = solution_df.merge(cell_data[['Cells', 'Rem_TimeToFreeze']], on='Cells')

# Calculate total freeze time for each pallet-cell allocation
solution_df['Total_Freeze_Time'] = solution_df['Pallet_TimeToFreeze'] + solution_df['Rem_TimeToFreeze']

# Identify empty or complete cells and cells with freezing in progress
empty_or_complete_cells = cell_data[cell_data['Rem_TimeToFreeze'] == 0]
freezing_in_progress_cells = cell_data[cell_data['Rem_TimeToFreeze'] != 0]

# Count and list these cells
num_empty_or_complete = len(empty_or_complete_cells)
num_freezing_in_progress = len(freezing_in_progress_cells)
list_empty_or_complete = empty_or_complete_cells['Cells'].tolist()
list_freezing_in_progress = freezing_in_progress_cells['Cells'].tolist()

# Current % of utilization
if num_empty_or_complete + num_freezing_in_progress > 0:
    current_utilization = num_freezing_in_progress / (num_empty_or_complete + num_freezing_in_progress) * 100
else:
    current_utilization = 0

# Display additional info before the main output
print("Number of cells empty or complete:", num_empty_or_complete)
print("List of cells empty or complete:", list_empty_or_complete)
print("\nNumber of cells with freezing-in-progress:", num_freezing_in_progress)
print("List of cells with freezing in progress:", [f"{cell} ({time} hours left)" for cell, time in zip(list_freezing_in_progress, freezing_in_progress_cells['Rem_TimeToFreeze'])])
print(f"Current % of utilization: {current_utilization:.2f}%")


Number of cells empty or complete: 197
List of cells empty or complete: ['L265123', 'L266121', 'L266122', 'L266123', 'L266124', 'L266125', 'L266126', 'L266131', 'L265141', 'L265143', 'L266142', 'L266144', 'L266145', 'L266146', 'L265151', 'L265152', 'L265153', 'L265154', 'L265155', 'L265156', 'L266151', 'L266152', 'L266153', 'L266154', 'L266155', 'L266156', 'L265161', 'L265162', 'L265165', 'L266161', 'L266162', 'L266163', 'L266164', 'L266165', 'L266166', 'L266171', 'L266172', 'L266173', 'L266174', 'L266176', 'L266181', 'L266182', 'L266183', 'L266184', 'L266185', 'L265191', 'L265192', 'L265193', 'L265194', 'L265195', 'L265196', 'L266191', 'L266192', 'L266193', 'L266194', 'L266195', 'L266196', 'L265202', 'L265206', 'L265212', 'L265215', 'L265221', 'L265222', 'L265223', 'L265224', 'L266222', 'L265231', 'L266233', 'L265242', 'L266242', 'L266243', 'L266244', 'L266245', 'L266246', 'L265255', 'L266253', 'L266256', 'L265261', 'L265262', 'L265263', 'L265264', 'L265265', 'L265266', 'L266261', 'L2

In [ ]:
# Display output
print("The number of pallets from load(s) on truck(s) =", len(pallet_data))
print("The number of cells in the freezer =", len(cell_data))
print("The minimum required time to freeze all pallets from load(s) on truck(s) by optimally allocating pallets into cells in the freezer =", ampl.getObjective('Time_Objective').value(), " hours")
print("\n")
print("ALLOCATION SCHEDULE: \n")
print("Optimal Pallet in Cell allocation is as follows:")

for idx, row in solution_df.iterrows():
    print(f"Place pallet {row['Pallets']} containing SKU {row['Pallet_SKU']} in cell {row['Cells']} at time {row['Rem_TimeToFreeze']} for {row['Pallet_TimeToFreeze']} for a total freeze time of {row['Total_Freeze_Time']}")


The number of pallets from load(s) on truck(s) = 200
The number of cells in the freezer = 288
The minimum required time to freeze all pallets from load(s) on truck(s) by optimally allocating pallets into cells in the freezer = 10370.21  hours


ALLOCATION SCHEDULE: 

Optimal Pallet in Cell allocation is as follows:
Place pallet 1 containing SKU 5025 in cell L265345 at time 0.0 for 46.0 for a total freeze time of 46.0
Place pallet 2 containing SKU 49538 in cell L265296 at time 0.0 for 43.75 for a total freeze time of 43.75
Place pallet 3 containing SKU 47898 in cell L266192 at time 0.0 for 53.5 for a total freeze time of 53.5
Place pallet 4 containing SKU 5710 in cell L265224 at time 0.0 for 41.5 for a total freeze time of 41.5
Place pallet 5 containing SKU 5528 in cell L266151 at time 0.0 for 36.25 for a total freeze time of 36.25
Place pallet 6 containing SKU 15017 in cell L266304 at time 0.0 for 54.75 for a total freeze time of 54.75
Place pallet 7 containing SKU 5000 in cell L266321